In [1]:
import pandas as pd
import tweepy
import os
from datetime import datetime

today = datetime.now().strftime("%y%m%d")

In [2]:
with open("../bearer_tokens.txt") as f:
    bearer_tokens = f.read().split("\n")
    
bearer_token = bearer_tokens[0]
client = tweepy.Client(bearer_token=bearer_token, wait_on_rate_limit=True)
auth = tweepy.OAuth2BearerHandler(bearer_token)
api = tweepy.API(auth, wait_on_rate_limit=True)

In [3]:
popular_accounts = pd.read_excel("Kadikoy-twitter-hesaplari-20220312.xlsx", sheet_name="popular-accounts")[["URL", "type"]]
accounts = pd.read_excel("Kadikoy-twitter-hesaplari-20220312.xlsx", sheet_name="accounts")[["url", "gender (male/female)"]].rename({"url": "URL", "gender (male/female)": "gender"}, axis=1)

In [4]:
accounts = accounts[accounts["URL"] != "https://twitter.com/search?q=kadıköylü&src=typed_query&f=top"]

In [5]:
popular_accounts["type"] = popular_accounts["type"].apply(lambda x: x.strip())
popular_accounts = popular_accounts.drop_duplicates().reset_index(drop=True)
popular_accounts["URL"].value_counts()

https://twitter.com/Kadikoy_Akademi    2
https://twitter.com/TAKadikoy          2
https://twitter.com/fisiltikadikoy     1
https://twitter.com/KadikoyTheatron    1
https://twitter.com/tvkadikoy          1
                                      ..
https://twitter.com/HDPKadikoyKadin    1
https://twitter.com/hdpkadikoy         1
https://twitter.com/serdildara         1
https://twitter.com/mustafaozarslnn    1
https://twitter.com/sahmettinozan      1
Name: URL, Length: 90, dtype: int64

In [6]:
popular_accounts[popular_accounts["URL"] == "https://twitter.com/Kadikoy_Akademi"]["type"]

67         basin-yayin
70    resmi-kurum-kisi
Name: type, dtype: object

In [7]:
popular_accounts[popular_accounts["URL"] == "https://twitter.com/TAKadikoy"]["type"]

6        mekan
11    topluluk
Name: type, dtype: object

In [8]:
popular_accounts = popular_accounts.drop(popular_accounts[(popular_accounts["URL"] == "https://twitter.com/Kadikoy_Akademi") & (popular_accounts["type"] == "basin-yayin")].index).reset_index(drop=True)
popular_accounts = popular_accounts.drop(popular_accounts[(popular_accounts["URL"] == "https://twitter.com/TAKadikoy") & (popular_accounts["type"] == "topluluk")].index).reset_index(drop=True)

In [9]:
accounts["gender"] = accounts["gender"].apply(lambda x: x.strip())
accounts = accounts.drop_duplicates().reset_index(drop=True)
accounts["URL"].value_counts()

https://twitter.com/fadimesisli        2
https://twitter.com/yaseminanderman    1
https://twitter.com/SerkanSil          1
https://twitter.com/Edbalist           1
https://twitter.com/zeynepakkus        1
                                      ..
https://twitter.com/can_ersal          1
https://twitter.com/essizerozge        1
https://twitter.com/ucukyesil          1
https://twitter.com/pturk222           1
https://twitter.com/leylaalp           1
Name: URL, Length: 115, dtype: int64

In [10]:
accounts[accounts["URL"] == "https://twitter.com/fadimesisli"]

,URL,gender
20,https://twitter.com/fadimesisli,female
47,https://twitter.com/fadimesisli,male


In [11]:
accounts = accounts.drop(accounts[(accounts["URL"] == "https://twitter.com/fadimesisli") & (accounts["gender"] == "male")].index).reset_index(drop=True)

In [12]:
popular_accounts["username"] = popular_accounts["URL"].apply(lambda x: x.split("/")[-1])
accounts["username"] = accounts["URL"].apply(lambda x: x.split("/")[-1])

In [13]:
usernames = sorted(list(pd.Series(list(popular_accounts["username"].unique()) + list(accounts["username"].unique())).unique()))
len(usernames)

204

In [14]:
users = []

for i in range(0, len(usernames), 100):
    user_data = client.get_users(usernames=usernames[i:i+99], user_fields=["created_at", "description", "entities", "id",
                                                                           "location", "name", "pinned_tweet_id",
                                                                           "profile_image_url", "protected", "public_metrics",
                                                                           "url", "username", "verified", "withheld"]).data
    users.extend(user_data)
    
users = [user.data for user in users]

In [15]:
for username in list(set(usernames).difference(set([user["username"] for user in users]))):
    try:
        user_data = client.get_user(username=username, user_fields=["created_at", "description", "entities", "id",
                                                                    "location", "name", "pinned_tweet_id",
                                                                    "profile_image_url", "protected", "public_metrics",
                                                                    "url", "username", "verified", "withheld"]).data.data
        users.append(user_data)
    except:
        pass

In [16]:
users_df = pd.DataFrame(users)

In [17]:
users_df["followers_count"] = users_df["public_metrics"].apply(lambda x: x["followers_count"])
users_df["following_count"] = users_df["public_metrics"].apply(lambda x: x["following_count"])
users_df["tweet_count"] = users_df["public_metrics"].apply(lambda x: x["tweet_count"])
users_df["listed_count"] = users_df["public_metrics"].apply(lambda x: x["listed_count"])
users_df = users_df.drop("public_metrics", axis=1)

users_df = users_df.drop_duplicates(subset=["username", "id"]).reset_index(drop=True)

In [18]:
users_df["followers_count"].astype(int).sum()

4208055

In [19]:
for i in range(0, len(users_df), 60):
    display(users_df.sort_values("followers_count", ascending=False).reset_index(drop=True).iloc[i:i+60][["username", "name", "location", "followers_count", "following_count", "tweet_count"]])

,username,name,location,followers_count,following_count,tweet_count
0,kadikoybelediye,Kadıköy Belediyesi,Hasanpaşa - KADIKÖY,2131688,12,69395
1,mkirikkanat,Mine G. Kırıkkanat,"İstanbul, Türkiye",493684,1177,143806
2,bengibaser,Prof. Dr. Bengi BAŞER,"İstanbul, Türkiye",419057,1667,27572
3,BuketUzuner,Buket Uzuner,Istanbul,175791,14359,108304
4,modasahnesi,moda sahnesi,Kadıköy - Bahariye Caddesi,114114,0,49144
5,serdildara,Av. Şerdil Dara Odabaşı,"Kadıköy, İstanbul",106269,172,4856
6,kadikoyemek,KadıköyEmekTiyatrosu,"İstanbul, Türkiye",83692,9,16579
7,ayse_cng,TC Ayşe Cng,NaN,44211,10782,181125
8,Av_SelamiOzturk,Av.Selami Öztürk,"Kadıköy, İstanbul",40386,976,12666
9,babasahne,Baba Sahne,"Kadıköy, İstanbul",40247,361,3963


,username,name,location,followers_count,following_count,tweet_count
60,mhp_kadikoy,MHP KADIKÖY,"Kadıköy, İstanbul",2598,223,13393
61,asabi_panda,asabi panda,Taksim,2576,668,18927
62,DoganDemetsezen,@DoğanDemetsezen🇹🇷🇹🇷,"İstanbul, Türkiye",2461,4779,27967
63,OykuSahne,ÖyküSahne,Kadıköy / Bahariye,2430,165,2610
64,fadimesisli,miraç,NaN,2422,5001,179680
65,kadikoykaymakam,Kadıköy Kaymakamlığı,"İstanbul, Türkiye",2388,157,4432
66,barisantik,Barış Antik,"Kadıköy, İstanbul",2294,1007,10921
67,buhangisarki,hümeysa,bakırköy ruh ve sinir hastanes,2230,299,17801
68,UrasAyse,Ayşe Uras,Turkey,2213,1438,5971
69,AklimdakiKdky,Aklımdaki Kadıköy,NaN,2189,589,1504


,username,name,location,followers_count,following_count,tweet_count
120,ekopol,Birgül,"İstanbul, Türkiye",624,286,17179
121,umarsiz,umarsiz,Çanakkale,612,616,37956
122,purkinjepurpurr,drpurkinje,NaN,611,963,246
123,ucukyesil,Guzide Erden,İstanbul,592,1207,25425
124,KatotKadikoy,Kadıköy Tiyatro Oyuncuları Topluluğu,"Kadıköy, İstanbul",586,242,227
125,Ahmet_Ozturk_,Ahmet Öztürk 🇹🇷,NaN,585,759,6789
126,cemerendemirr,eren,NaN,577,852,7495
127,mgezgor,Merhan Gezgör,"Kadıköy, İstanbul",563,1521,23069
128,aylinbulbull,aylin,İnstagram: @aaylinbulbull,560,193,11553
129,nebioglu_serdar,Serdar Nebioğlu,NaN,548,581,1026


,username,name,location,followers_count,following_count,tweet_count
180,kasgkizaihl,Kadıköy Ahmet Sani Gezici Kız AİHL,NaN,87,12,17
181,bayulken_kaan,Kaan Bayülken,NaN,87,714,2404
182,frhtyldzz,Ferhat Yıldızッ,NaN,85,405,6597
183,TugceCubukcu,Tuğçe Çubukçu,"Kadıköy, İstanbul",82,89,43
184,NurgulEl,Nurgül El,NaN,81,141,682
185,alabasburcin,Burcin Alabas,NaN,78,315,1719
186,tcizenel2,TC 🎼🌍🕊⚖️🌿☀️💦♻️🌌,PlanetHome-EarthİST.,60,290,13005
187,Muniseberk1,Munise berk kalan,"Kadıköy, İstanbul",58,193,1890
188,OzanUrcan,Ozan Urcan,"Sivas, Türkiye",56,250,68
189,ozlembahadr1,Özlem,NaN,51,515,299


In [20]:
user_counts = {users_df.sort_values("followers_count", ascending=False).reset_index(drop=True).loc[i, "username"]:users_df.sort_values("followers_count", ascending=False).reset_index(drop=True).loc[i, "followers_count"] for i in range(len(users_df))}
user_counts

{'kadikoybelediye': 2131688,
 'mkirikkanat': 493684,
 'bengibaser': 419057,
 'BuketUzuner': 175791,
 'modasahnesi': 114114,
 'serdildara': 106269,
 'kadikoyemek': 83692,
 'ayse_cng': 44211,
 'Av_SelamiOzturk': 40386,
 'babasahne': 40247,
 'DirenKadikoy1': 38806,
 'didemmcevik': 29787,
 'chabyhan': 27343,
 'MuzeGazhane': 24066,
 'leylaalp': 22623,
 'KadikoySahne': 19058,
 'gazetekadikoy': 18188,
 'KadikoySinemasi': 16668,
 'seyhanarman': 15702,
 'HdpKadikoy': 12389,
 'ayhanercan': 10411,
 'KadikoyKD': 10336,
 '58SA34': 10215,
 'kadikoychp': 8440,
 'TAKadikoy': 8409,
 'akkadikoy': 8362,
 'kadikoytiyatro': 8195,
 'CHP19Mayismah': 8183,
 'AtlasKadikoy': 8141,
 'Nogaylar1903': 8064,
 'kadikoyapartman': 7908,
 'mesutkosedagi': 7471,
 'pinardag': 7122,
 'yddayanisma': 7083,
 'desonsuzaaa': 5908,
 'dayanismaagii': 5875,
 'Pinozden': 5787,
 'semaklsn': 5704,
 'HaziranKadikoy': 5280,
 'alpaysalt': 4855,
 'pervinsomer': 4775,
 'kadikoykoop': 4651,
 'akgenckadikoy': 4508,
 'elektri': 4402,
 'akkad

In [21]:
if "follower_ids" not in os.listdir():
    os.mkdir("follower_ids")

In [22]:
kadikoy_accounts = list(users_df[users_df["protected"] == False].sort_values("followers_count")["username"])
print(len(kadikoy_accounts))

194


In [23]:
#pd.merge(users_df, pd.concat([accounts, popular_accounts]).reset_index(drop=True), on="username", how="outer").sort_values("followers_count", ascending=False).reset_index(drop=True)[["name", "username", "location", "gender", "type", "followers_count", "following_count", "tweet_count", "listed_count", "URL", "verified", "url", "id", "entities", "description", "profile_image_url", "pinned_tweet_id", "created_at", "protected"]].to_excel("Kadikoy-twitter-hesaplari-20220312_wMetadata.xlsx", index=False)

In [24]:
for i, account in enumerate(kadikoy_accounts[193:]):
    print(f"{i+1+193}- Collecting follower ids of {account} (# of followers: {user_counts[account]}) | {i+1+193}/{len(kadikoy_accounts)}")
    print("-"*60)
    
    page_num = 1
    error_count = 1
    follower_ids = []
    
    for page in tweepy.Cursor(api.get_follower_ids, screen_name=account).pages():
        try:
            print(f"Page: {page_num}")
            follower_ids.extend(page)
            print(f"Total number of follower ids collected: {len(follower_ids)}")
            page_num += 1
        except:
            print(f"Error in page: {page_num}")
            print(f"Error count: {error_count}")
            error_count += 1
            time.sleep(60)
            continue
        
    print(f"\nTotal number of follower ids collected for {account}: {len(follower_ids)}\n")
    
    with open(f"follower_ids/{today}_{account}-follower_ids.txt", "w+") as outfile:
        outfile.write("\n".join([str(follower_id) for follower_id in follower_ids]))
    print("-"*60)

194- Collecting follower ids of kadikoybelediye (# of followers: 2131688) | 194/194
------------------------------------------------------------
Page: 1
Total number of follower ids collected: 5000
Page: 2
Total number of follower ids collected: 10000
Page: 3
Total number of follower ids collected: 15000
Page: 4
Total number of follower ids collected: 20000
Page: 5
Total number of follower ids collected: 25000
Page: 6
Total number of follower ids collected: 30000
Page: 7
Total number of follower ids collected: 35000
Page: 8
Total number of follower ids collected: 40000
Page: 9
Total number of follower ids collected: 45000
Page: 10
Total number of follower ids collected: 50000
Page: 11
Total number of follower ids collected: 55000
Page: 12
Total number of follower ids collected: 60000
Page: 13
Total number of follower ids collected: 65000
Page: 14
Total number of follower ids collected: 70000
Page: 15
Total number of follower ids collected: 75000


Rate limit reached. Sleeping for: 892


Page: 16
Total number of follower ids collected: 80000
Page: 17
Total number of follower ids collected: 85000
Page: 18
Total number of follower ids collected: 90000
Page: 19
Total number of follower ids collected: 95000
Page: 20
Total number of follower ids collected: 100000
Page: 21
Total number of follower ids collected: 105000
Page: 22
Total number of follower ids collected: 110000
Page: 23
Total number of follower ids collected: 115000
Page: 24
Total number of follower ids collected: 120000
Page: 25
Total number of follower ids collected: 125000
Page: 26
Total number of follower ids collected: 130000
Page: 27
Total number of follower ids collected: 135000
Page: 28
Total number of follower ids collected: 140000
Page: 29
Total number of follower ids collected: 145000
Page: 30
Total number of follower ids collected: 150000


Rate limit reached. Sleeping for: 892


Page: 31
Total number of follower ids collected: 155000
Page: 32
Total number of follower ids collected: 160000
Page: 33
Total number of follower ids collected: 165000
Page: 34
Total number of follower ids collected: 170000
Page: 35
Total number of follower ids collected: 175000
Page: 36
Total number of follower ids collected: 180000
Page: 37
Total number of follower ids collected: 185000
Page: 38
Total number of follower ids collected: 190000
Page: 39
Total number of follower ids collected: 195000
Page: 40
Total number of follower ids collected: 200000
Page: 41
Total number of follower ids collected: 205000
Page: 42
Total number of follower ids collected: 210000
Page: 43
Total number of follower ids collected: 215000
Page: 44
Total number of follower ids collected: 220000
Page: 45
Total number of follower ids collected: 225000


...

Page: 391
Total number of follower ids collected: 1955000
Page: 392
Total number of follower ids collected: 1960000
Page: 393
Total number of follower ids collected: 1965000
Page: 394
Total number of follower ids collected: 1970000
Page: 395
Total number of follower ids collected: 1975000
Page: 396
Total number of follower ids collected: 1980000
Page: 397
Total number of follower ids collected: 1985000
Page: 398
Total number of follower ids collected: 1990000
Page: 399
Total number of follower ids collected: 1995000
Page: 400
Total number of follower ids collected: 2000000
Page: 401
Total number of follower ids collected: 2005000
Page: 402
Total number of follower ids collected: 2010000
Page: 403
Total number of follower ids collected: 2015000
Page: 404
Total number of follower ids collected: 2020000
Page: 405
Total number of follower ids collected: 2025000


Rate limit reached. Sleeping for: 891


Page: 406
Total number of follower ids collected: 2030000
Page: 407
Total number of follower ids collected: 2035000
Page: 408
Total number of follower ids collected: 2040000
Page: 409
Total number of follower ids collected: 2045000
Page: 410
Total number of follower ids collected: 2050000
Page: 411
Total number of follower ids collected: 2055000
Page: 412
Total number of follower ids collected: 2060000
Page: 413
Total number of follower ids collected: 2065000
Page: 414
Total number of follower ids collected: 2070000
Page: 415
Total number of follower ids collected: 2075000
Page: 416
Total number of follower ids collected: 2080000
Page: 417
Total number of follower ids collected: 2085000
Page: 418
Total number of follower ids collected: 2090000
Page: 419
Total number of follower ids collected: 2095000
Page: 420
Total number of follower ids collected: 2100000


Rate limit reached. Sleeping for: 892


Page: 421
Total number of follower ids collected: 2105000
Page: 422
Total number of follower ids collected: 2110000
Page: 423
Total number of follower ids collected: 2115000
Page: 424
Total number of follower ids collected: 2120000
Page: 425
Total number of follower ids collected: 2125000
Page: 426
Total number of follower ids collected: 2130000
Page: 427
Total number of follower ids collected: 2131676

Total number of follower ids collected for kadikoybelediye: 2131676

------------------------------------------------------------
